In [14]:
pip install --user psycopg2

  Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install --upgrade pip setuptools

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 2.0 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 2.0 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.3 MB 4.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.0/1.3 MB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 3.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstal

In [20]:
import psycopg2
import re

# Функция для подключения к базе данных с явной указанием кодировки
def connection():
    try:
        # Явно указываем кодировку для подключения
        conn = psycopg2.connect(
            host="localhost",        # Адрес сервера PostgreSQL
            database="suppliers",    # Имя базы данных
            user="postgres",         # Имя пользователя PostgreSQL
            password="acerAN",       # Пароль для подключения
            client_encoding="UTF8"   # Указание кодировки
        )
        return conn
    except psycopg2.OperationalError as e:
        print(f"Ошибка подключения к базе данных: {e}")
        exit(1)  # Завершаем программу, если не удалось подключиться

# Создание таблицы phonebook, если она еще не существует
def create_table():
    conn = connection()
    with conn.cursor() as cur:
        cur.execute("""
            CREATE TABLE IF NOT EXISTS phonebook (
                id SERIAL PRIMARY KEY,
                name VARCHAR(255),
                phone VARCHAR(20)
            );
        """)
        conn.commit()
    conn.close()
    print("Таблица phonebook создана или уже существует.")

# Вставка данных в таблицу
def insert_data():
    name = input("Введите имя: ")
    phone = input("Введите номер телефона: ")

    conn = connection()
    with conn.cursor() as cur:
        cur.execute("INSERT INTO phonebook (name, phone) VALUES (%s, %s);", (name, phone))
        conn.commit()
    conn.close()
    print("Данные успешно добавлены.")

# Получение всех записей из таблицы
def return_all():
    conn = connection()
    with conn.cursor() as cur:
        cur.execute("SELECT id, name, phone FROM phonebook ORDER BY name;")
        rows = cur.fetchall()
        for row in rows:
            print(row)
    conn.close()

# Удаление записи по имени или телефону
def delete_data():
    mode = input("Введите режим удаления (name/phone): ").strip().lower()

    conn = connection()
    with conn.cursor() as cur:
        if mode == "name":
            name = input("Введите имя для удаления: ")
            cur.execute("DELETE FROM phonebook WHERE name = %s;", (name,))
        elif mode == "phone":
            phone = input("Введите номер для удаления: ")
            cur.execute("DELETE FROM phonebook WHERE phone = %s;", (phone,))
        else:
            print("Неверный режим удаления.")
            conn.close()
            return

        conn.commit()
        print("Запись удалена.")
    conn.close()

# Проверка корректности номера телефона
def error_num(number):
    return re.fullmatch(r'(87\d{9}|\+77\d{9}|7\d{9})', number)

# Вставка нескольких пользователей с проверкой номеров
def many_users():
    conn = connection()
    with conn.cursor() as cur:
        num_of_users = int(input("Введите количество пользователей для добавления: "))
        errors = []

        for _ in range(num_of_users):
            name = input("Введите имя: ")
            phone = input("Введите номер телефона: ")

            if error_num(phone):
                cur.execute("INSERT INTO phonebook (name, phone) VALUES (%s, %s)", (name, phone))
            else:
                errors.append((name, phone))

        conn.commit()

        if errors:
            print("Некорректные номера:")
            for name, phone in errors:
                print(f"{name}: {phone}")
        else:
            print("Все номера корректны.")

    conn.close()

# Пагинация записей из таблицы
def limit_and_offset():
    limit = int(input("Введите лимит: "))
    offset = int(input("Введите смещение: "))

    conn = connection()
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM phonebook ORDER BY id LIMIT %s OFFSET %s;", (limit, offset))
        for row in cur.fetchall():
            print(row)
    conn.close()

# Вызов функций для тестирования
create_table()  # Создание таблицы
insert_data()   # Вставка одного контакта
return_all()    # Вывод всех записей
delete_data()   # Удаление записи
many_users()    # Вставка нескольких пользователей с проверкой номеров
limit_and_offset()  # Пагинация

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 61: invalid continuation byte